<a href="https://colab.research.google.com/github/tnfru/colab_research/blob/master/CIFAR10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import sys
if 'google.colab' in sys.modules:
  %tensorflow_version 2.x
import tensorflow as tf
from google.colab import drive
from sklearn.utils import shuffle
from keras.utils.np_utils import to_categorical
import copy

AUTO = tf.data.experimental.AUTOTUNE

## Set up TPU

In [31]:
try: 
  tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
except ValueError:
  tpu = None
  gpus = tf.config.experimental.list_logical_devices("GPU")

if tpu:
  tf.config.experimental_connect_to_cluster(tpu)
  tf.tpu.experimental.initialize_tpu_system(tpu)
  strategy = tf.distribute.experimental.TPUStrategy(tpu)
  print('Running on TPU ', tpu.master())  
elif len(gpus) > 0:
  strategy = tf.distribute.MirroredStrategy(gpus)
  print('Running on ', len(gpus), ' GPU(s) ')
else:
  strategy = tf.distribute.get_strategy()
  print('Running on CPU')

INFO:tensorflow:Initializing the TPU system: 10.80.21.178:8470


INFO:tensorflow:Initializing the TPU system: 10.80.21.178:8470


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Finished initializing TPU system.


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


Running on TPU  grpc://10.80.21.178:8470


## Read Data from Google Drive

In [32]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
PATH="/content/drive/My Drive/data/CIFAR10/"

In [0]:
def unpickle(file):
    import pickle
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict

In [0]:
db_1 = unpickle(PATH + "data_batch_1")
db_2 = unpickle(PATH + "data_batch_2")
db_3 = unpickle(PATH + "data_batch_3")
db_4 = unpickle(PATH + "data_batch_4")
db_5 = unpickle(PATH + "data_batch_5")
test_data = unpickle(PATH + "test_batch")
meta = unpickle(PATH + "batches.meta")

In [36]:
db_1[b'data'].shape, len(db_1[b'labels'])

((10000, 3072), 10000)

## Transform data

In [0]:
IMG_SIZE = [32, 32]
BATCH_SIZE = 128

In [38]:
X = np.append(db_1[b'data'], db_2[b'data'], axis=0)
X = np.append(X, db_3[b'data'], axis=0)
X = np.append(X, db_4[b'data'], axis=0)
X = np.append(X, db_5[b'data'], axis=0)
X = X.reshape(-1, *IMG_SIZE, 3)
X.shape

(50000, 32, 32, 3)

In [39]:
Y = copy.copy(db_1[b'labels'])
Y.extend(db_2[b'labels'])
Y.extend(db_3[b'labels'])
Y.extend(db_4[b'labels'])
Y.extend(db_5[b'labels'])
Y = np.array(Y)
Y = to_categorical(Y.reshape(-1, 1), num_classes=10)
Y.shape

(50000, 10)

In [40]:
X_test = test_data[b'data'].reshape(-1, *IMG_SIZE, 3)
Y_test = to_categorical(np.array(test_data[b'labels']).reshape(-1, 1), num_classes=10)
X_test.shape, Y_test.shape

((10000, 32, 32, 3), (10000, 10))

## Heal and normalize data

In [41]:
 np.isnan(X).sum()

0

In [0]:
X = X / 255
X_test = X_test / 255
X = X - X.mean(axis=0)
X_test = X_test - X_test.mean(axis=0)

In [43]:
X.mean()

-1.2150526510860496e-14

In [0]:
X = X / X.std(axis=0)
X_test = X_test / X_test.std(axis=0)

In [45]:
X.std(axis=0).sum() / (32 * 32 * 3)

1.000000000000001

## Convert Data to batches

In [0]:
def get_batched_dataset(dataset, train=False):
  if train:
    dataset = dataset.repeat()
    dataset = dataset.shuffle(1337)

  dataset = dataset.batch(BATCH_SIZE)
  dataset = dataset.prefetch(AUTO)
  return dataset

In [0]:
train_dataset = tf.data.Dataset.from_tensor_slices((X, Y))
test_dataset = tf.data.Dataset.from_tensor_slices((X_test, Y_test))

train = get_batched_dataset(train_dataset, train=True)
test = get_batched_dataset(test_dataset)

In [0]:
l = tf.keras.layers

In [0]:
def create_model():
  model = tf.keras.Sequential([
    l.Conv2D(kernel_size=3, filters=16, padding='same', activation='relu', input_shape=(*IMG_SIZE, 3)),
    l.BatchNormalization(),
    l.Conv2D(kernel_size=3, filters=32, padding='same', activation='relu'),
    l.BatchNormalization(),
    l.MaxPooling2D(),
    l.Dropout(0.5),

    l.Conv2D(kernel_size=3, filters=64, padding='same', activation='relu'),
    l.BatchNormalization(),
    l.Conv2D(kernel_size=3, filters=64, padding='same', activation='relu'),
    l.BatchNormalization(),
    l.MaxPooling2D(),
    l.Dropout(0.5),
    
    l.Conv2D(kernel_size=3, filters=128, padding='same', activation='relu'),
    l.BatchNormalization(),
    l.Conv2D(kernel_size=3, filters=128, padding='same', activation='relu'),
    l.BatchNormalization(),
    l.MaxPooling2D(),
    l.Dropout(0.5),

    l.Conv2D(kernel_size=3, filters=128, padding='same', activation='relu'),
    l.BatchNormalization(),
    l.Flatten(),
    l.Dense(10, activation='softmax')
  ])

  model.compile(
      optimizer='adam',
      loss='categorical_crossentropy',
      metrics=['accuracy']
  )

  return model

In [50]:
with strategy.scope():
  model = create_model()

  model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_7 (Conv2D)            (None, 32, 32, 16)        448       
_________________________________________________________________
batch_normalization_7 (Batch (None, 32, 32, 16)        64        
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 32, 32, 32)        4640      
_________________________________________________________________
batch_normalization_8 (Batch (None, 32, 32, 32)        128       
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 16, 16, 32)        0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 16, 16, 32)        0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 16, 16, 64)       

In [0]:
def lr_fun(x):
  return 1e-3 * 0.95 ** x

In [0]:
def display_training_curves(training, validation, title, subplot):
  #stolen from google cnn tutorial
  if subplot%10==1:
    plt.subplots(figsize=(10,10), facecolor='#F0F0F0')
    plt.tight_layout()
  ax = plt.subplot(subplot)
  ax.set_facecolor('#F8F8F8')
  ax.plot(training)
  ax.plot(validation)
  ax.set_title('model '+ title)
  ax.set_ylabel(title)
  ax.set_xlabel('epoch')
  ax.legend(['train', 'valid.'])

In [53]:
meta.keys()

dict_keys([b'num_cases_per_batch', b'label_names', b'num_vis'])

In [55]:
EPOCHS = 20

annealer = tf.keras.callbacks.LearningRateScheduler(lr_fun)
steps_per_epoch = X.shape[0] // BATCH_SIZE

history = model.fit(train, steps_per_epoch=steps_per_epoch, epochs=EPOCHS, validation_data=test, callbacks=[annealer], verbose=1)

InvalidArgumentError: ignored

In [0]:
display_training_curves(history.history['accuracy'], history.history['val_accuracy'], 'accuracy', 211)
display_training_curves(history.history['loss'], history.history['val_loss'], 'loss', 212)